#Titanic

Vi skal jobbe med det datasett som heter Titanic. I dette datasette har man en oversikt over alle mennesker som var på Titanic, hvor det er markert om en gitt person overlevde. Vi skal prøve å se om vi klarer å predikere om en person overlevde basert på data som vi har.

# Forberedelser

For å jobbe med maskinlæring må vi først forberede data til analyse.



## Last ned data

Først må vi laste ned data. Datasettet ligger i en csv-fil på nettet. Du kan lett laste den den med en linux-kommando:

In [0]:
!wget -q https://goo.gl/p7RMjN -O titanic_set.csv

Nå kan du se at under "Files" til venstre ligger det en fil "train.csv" (husk å trykke "Refresh"). Det er vårt datasett som vi skal jobbe videre med.

## Importere biblioteker

Vi må importere de nødvendige bibliotekene for å kunne jobbe videre med våre data.

`pandas` er et python-bibliotek som brukes til manipulering og analyse av data. 

`sklearn` står for scikit-learn og det er et stort python-bibliotek som brukes til maskinlæring

In [0]:
from sklearn import model_selection, metrics, svm
from sklearn.preprocessing import LabelEncoder
import pandas as pd

## Analysere og rense data

Vi kan starte med å lese data inn:

In [0]:
titanic = pd.read_csv("titanic_set.csv")

Vi kan også ta en titt på hvordan data er strukturert. Man kan, for eksempel, skrive ut de ti første radene.

In [0]:
print(titanic.head(10))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   
5            6         0       3   
6            7         0       1   
7            8         0       3   
8            9         1       3   
9           10         1       2   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   
5                                   Moran, Mr. James    male   NaN      0   
6                            McCarthy, Mr. Timothy J    male  54

Her ser vi at vi har informasjon om passasjerer - id, navn, kjønn, alder, hvilken lugar de hadde, hvor mye de betalte og mye mer. Vi kan også se hvem av dem som har overlevd ("Survived"). Det er denne informasjonen vi kan predikere ut ifra annen informasjon som vi har.

Vår fasit ligger altså i kolonnen "Survived". Vi kan lagre denne, og samtidig slette kolonnen fra datasettet for å kunne jobbe videre med data. La oss først se på formatet av fasiten vår:

In [0]:
labels = titanic["Survived"]
print(labels)

0      0
1      1
2      1
3      1
4      0
5      0
6      0
7      0
8      1
9      1
10     1
11     1
12     0
13     0
14     0
15     1
16     0
17     1
18     0
19     1
20     0
21     1
22     1
23     1
24     0
25     1
26     0
27     0
28     1
29     0
      ..
861    0
862    1
863    0
864    0
865    1
866    1
867    0
868    0
869    1
870    0
871    1
872    0
873    0
874    1
875    1
876    0
877    0
878    0
879    1
880    1
881    0
882    0
883    0
884    0
885    0
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


Og nå kan vi slette denne kolonnen:

In [0]:
del titanic["Survived"]

Vi kan skrive ut datasettet igjen for å sjekke hvordan det ser ut nå:

In [0]:
print(titanic.head(10))

   PassengerId  Pclass                                               Name  \
0            1       3                            Braund, Mr. Owen Harris   
1            2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2            3       3                             Heikkinen, Miss. Laina   
3            4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4            5       3                           Allen, Mr. William Henry   
5            6       3                                   Moran, Mr. James   
6            7       1                            McCarthy, Mr. Timothy J   
7            8       3                     Palsson, Master. Gosta Leonard   
8            9       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9           10       2                Nasser, Mrs. Nicholas (Adele Achem)   

      Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked  
0    male  22.0      1      0         A/5 21171   7.2500   NaN        S  
1  f

Vi trenger litt mer datatransformasjon for å analysere data. Vi bør bl.a. konvertere priser til hele verdier og normalisere dem:

In [0]:
le_priser = LabelEncoder()
le_priser.fit(titanic["Fare"])
titanic["Fare"] = le_priser.fit_transform(titanic["Fare"])
print(titanic.head(10))

Vi trenger ikke kolonner med manglende verdier, så disse kan vi bare slette:

In [0]:
titanic = titanic.dropna(axis=1)
print(titanic.head(10))

   PassengerId  Pclass                                               Name  \
0            1       3                            Braund, Mr. Owen Harris   
1            2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2            3       3                             Heikkinen, Miss. Laina   
3            4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4            5       3                           Allen, Mr. William Henry   
5            6       3                                   Moran, Mr. James   
6            7       1                            McCarthy, Mr. Timothy J   
7            8       3                     Palsson, Master. Gosta Leonard   
8            9       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9           10       2                Nasser, Mrs. Nicholas (Adele Achem)   

      Sex  SibSp  Parch            Ticket  Fare  
0    male      1      0         A/5 21171    18  
1  female      1      0          PC 17599   207  
2 

##Dele data opp

Til slutt må vi dele datasettet opp i test og trening, og etterpå er vi klare til maskinlæring!

`X_train` og `X_test` inneholder data som vi skal bruke for å predikere om en person overlevde, mens `y_train` og `y_test` inneholder fasiten (kolonnen Survived)

In [0]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(titanic._get_numeric_data(),
                                                                    labels, test_size=0.33,
                                                                    random_state=1)

# Maskinlæring

Vi starter med å lage en modell. I dette tilfellet har vi valgt å bruke SVM - support vector machine. Kort fortalt, representerer en SVM-modell alle elementer i treningssettet som punkter i et rom, kartlagt slik at eksempler på de forskjellige kategoriene er delt ved et gap. Nye eksempler blir satt inn i det samme rommet og kategorisert basert på hvilken side av gapet de havner. (Hunstad, Christian, and Daniel Reinholdt. Maskinlæring for predikering av live odds. MS thesis. NTNU, 2017)

In [0]:
classifier = svm.SVC(gamma=0.001, C=100.)
print(classifier)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


Så skal vi trene modellen. Vi bruker treningssettet `X_train` og fasiten `y_train`

In [0]:
classifier.fit(X_train, y_train)


SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Og da er vi klare til å predikere! Vi bruker testsettet for å teste vår classifier, og prøver å si for hver person i testsettet om h\*n overlevde eller ikke

In [0]:
prediksjoner = classifier.predict(X_test)

Det eneste som gjenstår nå, er å se hvor bra classifier-en klarte det. Da kan vi sammenligne våre prediksjoner med fasiten. Den enkleste måten å gjøre det på, er å se på *precision* og *recall*. **Precision** er antall relevante elementer blant alle elementer som er valgt, og **recall** er antall relevante elementer blant alle de valgte.



Man kan beregne precision og recall manuelt, ved å telle antall true positives, true negatives, false positives og false negatives, eller man kan også bruke noen innebygde rapporter i sklearn. De gir gjerne litt mer informasjon om statistikk - f1-score, micro, macro og weighted average. Du kan lese mer om den rapporten som vi skal bruke, [her](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

In [0]:
print("Resultater:")
print(metrics.classification_report(y_test, prediksjoner, target_names=["Nei", "Ja"]))

Resultater:
              precision    recall  f1-score   support

         Nei       0.62      0.64      0.63       174
          Ja       0.46      0.44      0.45       121

   micro avg       0.56      0.56      0.56       295
   macro avg       0.54      0.54      0.54       295
weighted avg       0.56      0.56      0.56       295



Som vi ser, er presisjonen ikke så høy - dette skyldes, mest sannsynlig, et litt for lite datasett. Allikevel, klarer algoritmen i 62% av tilfeller si riktig om en gitt person overlevde Titanic-turen.

Vi kan også ta en titt på confusion matrix - en måte å visualisere prestasjonen til algoritmen på. I en sånn matrise kan vi se hvor mange personer som faktisk overlevde, ble klassifisert som overlevde, og det motsatte - hvor mange døde var faktisk døde.

In [0]:
print("Confusion matrix:")
print(metrics.confusion_matrix(y_test, prediksjoner))


Confusion matrix:
[[112  62]
 [ 68  53]]
